#  Hiding Information in an Image Using Steganography

### A Project in Python 
This project demonstrates how to hide and retrieve text messages in an image using the Least Significant Bit (LSB) technique.

In [ ]:
from PIL import Image
import binascii


##  Step 1: Define Functions to Encode and Decode Text in Images

In [ ]:
def to_bin(data):
    if isinstance(data, str):
        return ''.join([format(ord(i), '08b') for i in data])
    elif isinstance(data, bytes) or isinstance(data, bytearray):
        return [format(i, '08b') for i in data]
    elif isinstance(data, int):
        return format(data, '08b')
    else:
        raise TypeError("Unsupported data type.")

def encode_image(img_path, message, output_path):
    image = Image.open(img_path)
    if image.mode != 'RGB':
        raise ValueError("Image mode must be RGB.")

    encoded = image.copy()
    width, height = image.size
    index = 0

    message += "###END###"
    message_bin = to_bin(message)
    message_len = len(message_bin)

    for row in range(height):
        for col in range(width):
            if index < message_len:
                r, g, b = image.getpixel((col, row))
                r_bin = to_bin(r)
                r = int(r_bin[:-1] + message_bin[index], 2)
                index += 1
                if index < message_len:
                    g_bin = to_bin(g)
                    g = int(g_bin[:-1] + message_bin[index], 2)
                    index += 1
                if index < message_len:
                    b_bin = to_bin(b)
                    b = int(b_bin[:-1] + message_bin[index], 2)
                    index += 1
                encoded.putpixel((col, row), (r, g, b))
            else:
                break
    encoded.save(output_path)
    print(f"✅ Message encoded and saved to {output_path}")


In [ ]:
def decode_image(img_path):
    image = Image.open(img_path)
    if image.mode != 'RGB':
        raise ValueError("Image mode must be RGB.")

    binary_data = ""
    for row in range(image.height):
        for col in range(image.width):
            r, g, b = image.getpixel((col, row))
            binary_data += to_bin(r)[-1]
            binary_data += to_bin(g)[-1]
            binary_data += to_bin(b)[-1]

    all_bytes = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    decoded_data = ""
    for byte in all_bytes:
        decoded_data += chr(int(byte, 2))
        if decoded_data[-8:] == "###END###":
            break
    return decoded_data[:-8]


##  Step 2: Encode a Message into an Image

In [ ]:
# Replace with your own image path
original_image = 'input_image.png'  # Make sure this file exists
output_image = 'encoded_image.png'
secret_message = "Hello, this is a secret message!"

encode_image(original_image, secret_message, output_image)


##  Step 3: Decode the Message from the Image

In [ ]:
decoded_message = decode_image('encoded_image.png')
print("🔓 Decoded Message:", decoded_message)


##  Conclusion
This project successfully demonstrates basic image steganography using LSB technique in Python.
You can enhance it further with encryption, audio/video steganography, or GUI applications.